In [1]:
# https://github.com/youngmihuang/lyrics_application
# https://pypi.org/project/GoogleNews/

from GoogleNews import GoogleNews

import requests 
from bs4 import BeautifulSoup
import pandas as pd

googlenews = GoogleNews()

googlenews.setlang('cn')
googlenews.setperiod('d')
googlenews.setencode('utf-8')
googlenews.clear()


x = input("請輸入要搜尋的關鍵字，將為你搜集相關字詞內容:")
googlenews.search(x)

alldata = googlenews.result()
result = googlenews.gettext()
links = googlenews.get_links()
# print(type(result))
# print(len(result))
# print(alldata)


print()

for n in range(len(result)):
#     print(n+1)
    print(result[n])
    print(links[n])
   

df = pd.DataFrame(
{
    '標題': result,
    '連結': links
})
 
df



高雄城中城大火 副總統賴清德盼建築業構思更好更安全設計
https://udn.com/news/story/122505/5834280?from=udn-ch1_breaknews-1-cate2-news
城中城大火副總統盼建築業構思更好更安全設計| 政治
https://www.cna.com.tw/news/aipl/202110210373.aspx
快帶我回家！「倖存小貓」逃過城中城大火…網心疼集氣：希望 ...
https://tw.news.yahoo.com/%E5%BF%AB%E5%B8%B6%E6%88%91%E5%9B%9E%E5%AE%B6-%E5%80%96%E5%AD%98%E5%B0%8F%E8%B2%93-%E9%80%83%E9%81%8E%E5%9F%8E%E4%B8%AD%E5%9F%8E%E5%A4%A7%E7%81%AB-%E7%B6%B2%E5%BF%83%E7%96%BC%E9%9B%86%E6%B0%A3-%E5%B8%8C%E6%9C%9B%E9%A3%BC%E4%B8%BB%E9%82%84%E5%9C%A8-133148973.html
怕了城中城！彰化清查近百棟大樓驚見9成不合格｜ 蘋果新聞網 ...
https://tw.appledaily.com/local/20211021/UMTX7ZCEQVGGFAX5BJRMSYTPU4/
城中城經查並無依法定成程序成立管委會-社會
https://times.hinet.net/news/23564798
城中城大火明公祭陳其邁：以人為中心全力協助安置、法律等後續 ...
https://times.hinet.net/news/23564799
哀悼城中城罹難者法國駐台代表到靈堂致意- 社會
https://news.ltn.com.tw/news/society/breakingnews/3711865
城中城大火明公祭陳其邁：以人為中心全力協助安置、法律等後續 ...
https://news.sina.com.tw/article/20211021/40300306.html
藍議員提證明「城中城有管委會」工務局：未合法化
https://tw.news.yahoo.com/%E8%97%8D%E8%AD%B0%E5%93%A1%E6%8F%90%

,標題,連結
0,高雄城中城大火 副總統賴清德盼建築業構思更好更安全設計,https://udn.com/news/story/122505/5834280?from...
1,城中城大火副總統盼建築業構思更好更安全設計| 政治,https://www.cna.com.tw/news/aipl/202110210373....
2,快帶我回家！「倖存小貓」逃過城中城大火…網心疼集氣：希望 ...,https://tw.news.yahoo.com/%E5%BF%AB%E5%B8%B6%E...
3,怕了城中城！彰化清查近百棟大樓驚見9成不合格｜ 蘋果新聞網 ...,https://tw.appledaily.com/local/20211021/UMTX7...
4,城中城經查並無依法定成程序成立管委會-社會,https://times.hinet.net/news/23564798
5,城中城大火明公祭陳其邁：以人為中心全力協助安置、法律等後續 ...,https://times.hinet.net/news/23564799
6,哀悼城中城罹難者法國駐台代表到靈堂致意- 社會,https://news.ltn.com.tw/news/society/breakingn...
7,城中城大火明公祭陳其邁：以人為中心全力協助安置、法律等後續 ...,https://news.sina.com.tw/article/20211021/4030...
8,藍議員提證明「城中城有管委會」工務局：未合法化,https://tw.news.yahoo.com/%E8%97%8D%E8%AD%B0%E...
9,用鏡頭看台灣》城中城巨災殷鑑不遠北市消防救災演練超逼真 ...,https://www.chinatimes.com/realtimenews/202110...


In [2]:

url = df['連結'][0]
print(url)


user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}


r = requests.get(url, headers=user_agent)
r.encoding = "utf-8"
web_content = r.text
soup = BeautifulSoup(web_content,'html')


articleContent = soup.find_all('p')
# articleContent

article = []
for p in articleContent:
    article.append(p.text)


articleAll = '\n'.join(article)
# print(articleAll)

# 分段用

https://udn.com/news/story/122505/5834280?from=udn-ch1_breaknews-1-cate2-news


In [3]:

import jieba
jieba.load_userdict('dict.txt.big.txt')

d = articleAll.replace('[^\w\s]','').replace('／',"").replace('《','').replace('》','').replace('，','').replace('。','').replace('「','').replace('」','').replace('（','').replace('）','').replace('！','').replace('？','').replace('、','').replace('▲','').replace('…','').replace('：','')
# print(d)

jieba.setLogLevel(20)

# Sentence = jieba.cut(d, cut_all=True)
# print('全模式'+": "  + "/ ".join(Sentence) + '\n')   
 
# Sentence = jieba.cut(d, cut_all=False)
# print('精確模式'+": " + "/ ".join(Sentence)+ '\n')  
 
# Sentence = jieba.cut(d)  
# print('Default為精確模式'+": " + "/ ".join(Sentence)+ '\n')
 
Sentence = jieba.cut_for_search(d)  
print('搜索引擎模式'+": " + "/ ".join(Sentence))
print(Sentence)


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/2p/qz_90rqs1fx0_vgnt2xgf6yw0000gn/T/jieba.cache
Loading model cost 1.706 seconds.
Prefix dict has been built successfully.


搜索引擎模式: 副/ 總統/ 賴清德/ 今天/ 出席/ 第/ 23/ 屆國家/ 建築/ 金質獎/ 他/ 表示/ 高雄/ 城中/ 中城/ 城中城/ 發生/ 大火/ 奪/ 走/ 46/ 條寶貴/ 人命/ ；/ 雖然/ 致命/ 未必/ 與/ 建築/ 設計/ 有/ 關不過/ 盼/ 建築界/ 能夠/ 從/ 不幸/ 事件/ 中記取/ 教訓/ 並構/ 思出/ 更好/ 更/ 安全/ 的/ 設計/ 
/ 
/ 第/ 23/ 屆國家/ 建築/ 金質獎/ 暨/ 第/ 18/ 屆國家/ 品牌/ 玉山/ 獎頒獎/ 典禮/ 今天/ 登場/ 賴清德致/ 詞時/ 表示/ 國家/ 建築/ 金質獎/ 得主/ 是/ 讓/ 世界/ 看見/ 台灣/ 讓/ 台灣/ 走進/ 世界/ 的/ 最佳/ 推手/ 得獎/ 作品/ 不管/ 是/ 公私/ 建築/ 中央/ 或/ 地方/ 都/ 看/ 得到/ 得/ 獎者/ 在/ 環境/ 永續/ 友善/ 智慧/ 以人/ 為/ 本/ 等/ 的/ 巧思/ 且/ 符合/ 建築/ 發展/ 潮流/ 
/ 
/ 賴清德/ 表示/ 近期/ 高雄/ 城中/ 中城/ 城中城/ 發生/ 大火/ 奪/ 走/ 46/ 條寶貴/ 人命/ 雖然/ 致命/ 原因/ 很多/ 不/ 一定/ 和/ 建築/ 設計/ 有/ 關不過/ 盼/ 建築界/ 能夠/ 從/ 不幸/ 事件/ 中/ 抽絲/ 剝繭/ 記取/ 教訓/ 不用/ 法令/ 規範/ 就/ 能/ 構思出/ 更好/ 更/ 安全/ 的/ 設計/ 並且/ 從/ 建築/ 專業/ 角度/ 降低/ 天災/ 人禍/ 對民眾/ 生命/ 財產/ 安全/ 的/ 威脅/ 
/ 
/ 他/ 說/ 台灣/ 目前/ 面臨/ 都市/ 熱島/ 效應/ 能源/ 轉型/ 高齡化/ 社會/ 及/ 危老/ 建築/ 等/ 多項/ 問題/ 期盼/ 業界/ 能夠/ 與/ 政府/ 一起/ 合作/ 攜/ 手推/ 動綠能/ 產業/ 打造/ 健康/ 綠建築/ 提升/ 建築/ 安全/ 的/ 生活/ 環境/ 
/ 
/ 賴清德/ 指出/ 建築業/ 是/ 各行/ 各業/ 的/ 火車/ 頭產業/ 從/ 建築/ 方面/ 開始/ 導入/ 創新節/ 能/ 工法/ 建材/ 與/ 設備/ 將對/ 相關/ 產業/ 發揮/ 示範/ 作用/ 期盼/ 未來/ 新建/ 築能/ 運用/ 專業/ 技術/ 融入/ 綠能/ 與/ 儲能/ 系統/ 加快/ 

In [4]:
import numpy as np

from PIL import Image

from collections import Counter

import matplotlib.pyplot as plt

from wordcloud import WordCloud, ImageColorGenerator

from scipy.ndimage import gaussian_gradient_magnitude

with open('stopword.txt', 'r', encoding="utf-8") as f:
    stopwords = f.read().split('\n')

terms = {}
for sentence in Sentence:
    if sentence in stopwords:
        continue

    if sentence in terms:
        terms[sentence] += 1
    else:
        terms[sentence] = 1

print(Counter(terms))
# generate_from_text()方法會採納stopwords參數 也可使用

# https://coolors.co/palettes/popular


artDf = pd.DataFrame.from_dict(terms, orient='index', columns=['詞頻'])
artDf.sort_values(by=['詞頻'], ascending=False)


img = "color"
img_path = "%s.png" % img

mask_color = np.array(Image.open(img_path))
mask_color = mask_color[::3, ::3]
mask_image = mask_color.copy()
mask_image[mask_image.sum(axis=2) == 0] = 255

edges = np.mean([gaussian_gradient_magnitude(mask_color[:, :, i] / 255., 2) for i in range(3)], axis=0)
mask_image[edges > .08] = 255

wc = WordCloud(font_path="/Users/larry/Library/Fonts/SourceHanSansTW-Regular.otf",
               mask=mask_color,
               max_font_size=45,
               max_words=2000,
               stopwords=stopwords,
               margin=1,
               relative_scaling=0)

wc.generate_from_frequencies(terms)
image_colors = ImageColorGenerator(mask_color)
wc.recolor(color_func=image_colors)

# 視覺化

plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.figure(figsize=(25, 25))
plt.show()

# plt.savefig("Wordcloud.png")
wc.to_file("img/2021-1021-城中城.png")


Counter()


ValueError: We need at least 1 word to plot a word cloud, got 0.

In [ ]:

artDf = pd.DataFrame.from_dict(dicition, orient='index', columns=['詞頻'])
Frequence = artDf.sort_values(by=['詞頻'], ascending=False).head(50)
print(Frequence)